In [31]:
import numpy as np
import pandas as pd


def load_expression_data(file_name,drop):
    '''
    input: gene expression filename
    output: pandas dataframe
    '''
    df = pd.read_csv(file_name,sep='\t')
    if drop:
        df = df.drop(columns='SYMBOL')
    #print(df.head())
    return df

def get_expression_genes(df):
    '''
    input: dataframe
    output: list of genes, 10712
    '''
    df_small = df.iloc[:,df.columns=="SYMBOL"]
    print(df_small.shape)
    print(df_small.head())
    gene_list = df_small.to_numpy().flatten()
    print(len(gene_list),gene_list.shape)
    return gene_list 

def load_samples(file_name):
    '''
    input: filename of samples
    output: distionary, key=patient name, value=0 healthy, 1 diseased
    '''
    d = {}
    with open(file_name, "r") as fh:
        lines = fh.readlines()
        for line in lines:
            tokens = line.split()
            d[tokens[0]] = tokens[1]
    #print("d:",d)
    return d


    
def load_kegg(filename,expr_gene):
    '''
    input:kegg file name
    output: dictionary key=KEGG_CITRATE_CYCLE_TCA_CYCLE value=list of genes which are in expression data
    '''
    path_gene = {}
    with open(filename, "r") as fh:
        lines = fh.readlines()
        
        for i in range(0,len(lines)):
            tokens = lines[i].split()
            #print("pathway:",tokens[0])
            genes = tokens[2:]
            if i==0:
                print("lines[i]:",lines[i])
                print("tokens[2:]",tokens[2:],len(tokens[2:]))
            #print("pathway genes:",genes,"num pathway genes:",len(genes))
            g_list=[]
            for g in genes:
                if g in expr_gene:
                    g_list.append(g)
            #print("num genes:",len(g_list),g_list)
            path_gene[tokens[0]] = g_list
            #print("g_list:",len(g_list),g_list)
        return path_gene

def load_samples(file_name):
    '''
    input: filename of samples
    output: distionary, key=patient name, value=0 healthy, 1 diseased
    '''
    d = {}
    with open(file_name, "r") as fh:
        lines = fh.readlines()
        for line in lines:
            tokens = line.split()
            d[tokens[0]] = tokens[1]
    #print("d:",d)
    return d
#1) samp dict, df expression data, expr_gene pathway,gene dict. 
samp = load_samples("GSE25628_samples.txt")
df = load_expression_data("GSE25628_filtered_expression.txt",False)       
expr_gene = get_expression_genes(df)
print(expr_gene)
pathway_gene_dict = load_kegg("c2.cp.kegg.v6.2.symbols.filtered.gmt",expr_gene)

#print(pathway_gene_dict)
print(pathway_gene_dict['KEGG_GLYCOLYSIS_GLUCONEOGENESIS'],len(pathway_gene_dict['KEGG_GLYCOLYSIS_GLUCONEOGENESIS']))

print(df.head())

rows,cols = df.shape
#build healthy and diseased indexes
print(df.columns)
healthy_idx=[]
disease_idx=[]
for idx,col in enumerate(df.columns):
    if col!="SYMBOL":
        print("idx:",idx,"col:",col,samp[col])
        if samp[col]=='1':
            disease_idx.append(idx)
        elif samp[col]=='0':
            healthy_idx.append(idx)
        else:
            print("xxxx")
    
print("healthy_idx:",healthy_idx," disease_idx:",disease_idx)
df_np = df.to_numpy()
print("rows:",rows,"cols:",cols)#fetch healthy as list of columns, diseased as list of columns
rank_me=[]
healthy_mean=np.mean(df_np[0,[healthy_idx]])
disease_mean=np.mean(df_np[0,[disease_idx]])
for idx in range(0,df_np.shape[0]):
    #print("df iloc:",df.iloc[idx,0],df_np[0,[healthy_idx]],healthy_mean)
    #print(df_np[0,[disease_idx]],disease_mean)
    #print("diff:",disease_mean-healthy_mean)
    healthy_mean=np.mean(df_np[idx,[healthy_idx]])
    disease_mean=np.mean(df_np[idx,[disease_idx]])
    rank_me.append((idx,df.iloc[idx,0],disease_mean-healthy_mean)) #sort by second tuple value

#before sorting
print("rank_me[0:5]",rank_me[0:5])
largest_first = sorted(rank_me, key = lambda x: x[2],reverse=True)
print("rank_me[0:5]",largest_first[0:5])
return_me=[]
for x in largest_first:
    return_me.append(x[1])
print(return_me[0:5])

#calculate ES for each geneset using ranked list. wow for all 10k? 
#

#for idx in range(0,df_np.shape[0]):
#    print(df_np[idx,[healthy_idx]])
#    print(df_np[idx,[disease_idx]])
    
#def q12()
#def q13()
#def q14()

(10702, 1)
   SYMBOL
0     A2M
1  A4GALT
2    AAAS
3    AACS
4   AAGAB
10702 (10702,)
['A2M' 'A4GALT' 'AAAS' ... 'ZYX' 'ZZEF1' 'ZZZ3']
lines[i]: KEGG_GLYCOLYSIS_GLUCONEOGENESIS	http://www.broadinstitute.org/gsea/msigdb/cards/KEGG_GLYCOLYSIS_GLUCONEOGENESIS	ACSS2	GCK	PGK2	PGK1	PDHB	PDHA1	PDHA2	PGM2	TPI1	ACSS1	FBP1	ADH1B	HK2	ADH1C	HK1	HK3	ADH4	PGAM2	ADH5	PGAM1	ADH1A	ALDOC	ALDH7A1	LDHAL6B	PKLR	LDHAL6A	ENO1	PKM2	PFKP	BPGM	PCK2	PCK1	ALDH1B1	ALDH2	ALDH3A1	AKR1A1	FBP2	PFKM	PFKL	LDHC	GAPDH	ENO3	ENO2	PGAM4	ADH7	ADH6	LDHB	ALDH1A3	ALDH3B1	ALDH3B2	ALDH9A1	ALDH3A2	GALM	ALDOA	DLD	DLAT	ALDOB	G6PC2	LDHA	G6PC	PGM1	GPI

tokens[2:] ['ACSS2', 'GCK', 'PGK2', 'PGK1', 'PDHB', 'PDHA1', 'PDHA2', 'PGM2', 'TPI1', 'ACSS1', 'FBP1', 'ADH1B', 'HK2', 'ADH1C', 'HK1', 'HK3', 'ADH4', 'PGAM2', 'ADH5', 'PGAM1', 'ADH1A', 'ALDOC', 'ALDH7A1', 'LDHAL6B', 'PKLR', 'LDHAL6A', 'ENO1', 'PKM2', 'PFKP', 'BPGM', 'PCK2', 'PCK1', 'ALDH1B1', 'ALDH2', 'ALDH3A1', 'AKR1A1', 'FBP2', 'PFKM', 'PFKL', 'LDHC', 'GAPDH', 'ENO3', 'ENO2', 'PGAM4', 

In [ ]:
f